In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=628f5c15a62a18f34b9e936b00824e7a87b3b2b3838c0e9638377e2a6bc0469e
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
from google.colab import drive
import os
import cv2
from google.colab.patches import cv2_imshow
import datetime
import csv
import pandas as pd
import face_recognition

In [3]:
# drive.mount('/content/drive', force_remount=True)
# input_folder = '/content/drive/MyDrive/segmentation_to_process'
# output_folder = '/content/drive/MyDrive/segmentation_output'

# now = datetime.datetime.now()
# now.isoformat()
# directory = str(now)

# dest_path = os.path.join(output_folder, directory)
# archive_path = os.path.join(dest_path, "source_files")

# os.mkdir(dest_path)
# os.mkdir(archive_path)
# metadata_path = os.path.join(dest_path, 'metadata.csv')
# header = ['title', 'year', 'issue', 'page', 'filename']

# with open(metadata_path, 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)
now = datetime.datetime.now()
now.isoformat()
directory = str(now)
input_folder = 'yearbook'
output_folder = 'yearbook_output'
dest_path = os.path.join(output_folder, directory)
archive_path = os.path.join(dest_path, "source_files")

In [23]:
def segment(file,df):
  filebase = os.path.splitext(file)[0]
  try:
    title, year, issue, page,_ = filebase.split('_')
  except:
    print(filebase)
  imagepath = os.path.join(input_folder, file)
  findFaces = face_recognition.load_image_file(imagepath)
  encodings = face_recognition.face_encodings(findFaces)
  if len(encodings) > 0:
    suffix= ".jpg"
    locations = face_recognition.face_locations(findFaces)
    for (top, right, bottom, left) in locations: #we can turn this into multiple little pictures if thats more convinient later
      cv2.rectangle(findFaces, (left, top), (right, bottom), (0, 255, 0), 2)
    dest_filepath = os.path.join(output_folder, filebase + suffix)
    cv2.imwrite(dest_filepath + '.jpg', findFaces)
    cv2.imwrite(dest_filepath, findFaces)
    data=[title, year, issue, page, filebase,encodings]
    df.loc[len(df)] = data


In [24]:
file_list = [file for file in os.listdir(input_folder) if file.lower().endswith('.jpg')]
print(len(file_list), 'files found')

df = pd.DataFrame(columns=['title', 'year', 'issue', 'page', 'filename','encodings'])

for file in file_list:
    segment(file,df)
    source_image_path = os.path.join(input_folder, file)
df.to_csv(os.path.join(input_folder,'metadata.csv'), index=False)

45 files found


In [26]:
df.head(10)

,title,year,issue,page,filename,encodings
0,AUC yearbook,2003,1,5,AUC yearbook_2003_1_5_4,"[[-0.09035027027130127, 0.1378680020570755, 0...."
1,AUC yearbook,2003,1,42,AUC yearbook_2003_1_42_5,"[[-0.04943913221359253, -0.0035008415579795837..."
2,AUC yearbook,1958,1,31,AUC yearbook_1958_1_31_1,"[[-0.10619190335273743, -0.051437437534332275,..."
3,AUC yearbook,1958,1,7,AUC yearbook_1958_1_7_3,"[[-0.027216769754886627, 0.17832452058792114, ..."
4,AUC yearbook,1980,1,10,AUC yearbook_1980_1_10_2,"[[-0.0883668065071106, -0.005980312824249268, ..."
5,AUC yearbook,2003,1,5,AUC yearbook_2003_1_5_3,"[[-0.044776760041713715, 0.0778794214129448, 0..."
6,AUC yearbook,1980,1,10,AUC yearbook_1980_1_10_12,"[[-0.06744514405727386, 0.08748582750558853, 0..."
7,AUC yearbook,1980,1,6,AUC yearbook_1980_1_6_2,"[[-0.04063311964273453, 0.05413972586393356, 0..."
8,AUC yearbook,1960,1,19,AUC yearbook_1960_1_19_1,"[[-0.054895833134651184, 0.010994940996170044,..."
9,AUC yearbook,1980,1,10,AUC yearbook_1980_1_10_1,"[[-0.07778427004814148, 0.014534316956996918, ..."


In [32]:
import shutil
shutil.make_archive('yearbook', 'zip', 'yearbook')

'/content/yearbook.zip'

In [33]:
shutil.make_archive('yearbook_output', 'zip', 'yearbook_output')

'/content/yearbook_output.zip'